<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_5SSC_Experiment/blob/main/SleepStage_EEG_EXperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Function

In [ ]:
!pip install imbalanced-learn
!pip install ipywidgets


In [2]:
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [3]:
def ddataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)

def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic


def models_check_box(models):
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys


def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)


def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

#Starting

In [63]:
dataset = pd.read_csv("https://raw.githubusercontent.com/RafsanJany-44/Reopen_5_stage_sleep_classifiacation/main/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

(108920, 52)
(108450, 52)


In [64]:
dataset.head()

,Subject,Sleep Stage,Epoch,MeanP_Alpha_F4,MeanP_Beta_F4,MeanP_Theta_F4,MeanP_Delta_F4,MeanP_Gamma_F4,MeanP_Alpha_C4,MeanP_Beta_C4,...,DAR_C4,DAR_O2,DAR_G,DTR_F4,DTR_O2,DTABR_F4,DTABR_C4,DTABR_O2,DTABR_G,P_G
0,SN002,W,0,0.00051,0.00071,0.00121,0.01070,0.000240,0.00041,0.00067,...,8.61,25.00,18.20,8.85,18.60,9.77,3.83,8.67,7.42,0.00918
1,SN002,W,1,0.00040,0.00051,0.00062,0.00539,0.000120,0.00038,0.00052,...,6.26,30.10,16.60,8.69,28.90,6.60,3.16,12.30,7.34,0.00636
2,SN002,W,2,0.00036,0.00047,0.00059,0.00967,0.000100,0.00036,0.00049,...,8.67,10.50,15.30,16.40,9.58,12.40,4.19,4.62,7.06,0.00628
3,SN002,W,3,0.00035,0.00047,0.00043,0.00228,0.000110,0.00036,0.00053,...,4.00,3.64,4.72,5.30,3.64,3.30,2.07,1.90,2.42,0.00278
4,SN002,W,4,0.00033,0.00043,0.00053,0.00436,0.000094,0.00031,0.00043,...,4.81,7.61,8.54,8.23,7.29,6.43,2.53,3.49,4.15,0.00369


###Enocoding

In [65]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

###Spliting into X and y

In [66]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]

X.head(10)

,MeanP_Alpha_F4,MeanP_Beta_F4,MeanP_Theta_F4,MeanP_Delta_F4,MeanP_Gamma_F4,MeanP_Alpha_C4,MeanP_Beta_C4,MeanP_Theta_C4,MeanP_Delta_C4,MeanP_Gamma_C4,...,DAR_C4,DAR_O2,DAR_G,DTR_F4,DTR_O2,DTABR_F4,DTABR_C4,DTABR_O2,DTABR_G,P_G
0,0.00051,0.00071,0.00121,0.01070,0.000240,0.00041,0.00067,0.00061,0.00353,0.000210,...,8.61,25.00,18.20,8.85,18.60,9.77,3.83,8.67,7.42,0.00918
1,0.00040,0.00051,0.00062,0.00539,0.000120,0.00038,0.00052,0.00046,0.00238,0.000100,...,6.26,30.10,16.60,8.69,28.90,6.60,3.16,12.30,7.34,0.00636
2,0.00036,0.00047,0.00059,0.00967,0.000100,0.00036,0.00049,0.00044,0.00312,0.000087,...,8.67,10.50,15.30,16.40,9.58,12.40,4.19,4.62,7.06,0.00628
3,0.00035,0.00047,0.00043,0.00228,0.000110,0.00036,0.00053,0.00040,0.00144,0.000110,...,4.00,3.64,4.72,5.30,3.64,3.30,2.07,1.90,2.42,0.00278
4,0.00033,0.00043,0.00053,0.00436,0.000094,0.00031,0.00043,0.00038,0.00149,0.000081,...,4.81,7.61,8.54,8.23,7.29,6.43,2.53,3.49,4.15,0.00369
5,0.00036,0.00043,0.00038,0.00194,0.000085,0.00039,0.00048,0.00041,0.00100,0.000081,...,2.56,2.44,3.47,5.11,2.64,2.94,1.62,1.52,2.02,0.00238
6,0.00034,0.00047,0.00041,0.00159,0.000100,0.00038,0.00054,0.00048,0.00138,0.000100,...,3.63,4.87,4.39,3.88,4.48,2.47,2.02,2.40,2.30,0.00260
7,0.00035,0.00048,0.00040,0.00128,0.000094,0.00041,0.00064,0.00051,0.00134,0.000110,...,3.27,3.04,3.32,3.20,2.59,2.02,1.76,1.76,1.85,0.00240
8,0.00035,0.00042,0.00052,0.00134,0.000094,0.00043,0.00047,0.00078,0.00156,0.000087,...,3.63,3.08,3.51,2.58,2.14,2.42,2.60,2.02,2.34,0.00260
9,0.00050,0.00053,0.00077,0.00290,0.000087,0.00067,0.00073,0.00139,0.00549,0.000095,...,8.19,4.81,6.27,3.77,2.77,3.56,4.91,2.95,3.81,0.00520


###USing SMOTE for balancing

In [ ]:
new_dataset =  balance(X,y)

In [ ]:
new_dataset[target].value_counts()

4    39428
0    39428
1    39428
2    39428
3    39428
Name: Sleep Stage, dtype: int64

#Feature Selection

In [ ]:
number_of_feat = 30

###ANOVA with f classifciation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])

###Embedded Method

In [ ]:
''''
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
'''

###Pearson's with f regression

In [ ]:
'''
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(dataset.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
'''

###Sequential Feature Selection

In [ ]:
''''
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
  '''

###Feature list

In [ ]:
dfcolumns = pd.DataFrame(list_of_feat)
print(dfcolumns)

#Data Spliting Into X,y

In [67]:
#X_new = new_dataset[list_of_feat]
#y_new = new_dataset[target]

#X_new =  new_dataset.loc[:,new_dataset.columns != target] 
#y_new = new_dataset[target]

X_new = X
y_new = y

#X_new.head(10)
print(X_new.shape)

(108450, 49)


#Data Scaling

In [ ]:
X_new =  StandardScaleData(X_new)
print(X_new.shape)
X_new

(108450, 49)


array([[-0.0653349 , -0.06741043, -0.04805037, ..., -0.11649183,
        -0.11649183, -0.12025476],
       [-0.07426116, -0.09237015, -0.06824147, ..., -0.11649183,
        -0.11649183, -0.13409893],
       [-0.07750708, -0.0973621 , -0.06926814, ..., -0.11649183,
        -0.11649183, -0.13449167],
       ...,
       [-0.07426116, -0.10984196, -0.07269036, ..., -0.11649183,
        -0.11649183, -0.15363786],
       [-0.0653349 , -0.06241848, -0.06892592, ..., -0.11649183,
        -0.11649183, -0.13620992],
       [-0.00285108,  0.22337036, -0.02272594, ..., -0.11649183,
        -0.11649183, -0.03915347]])

#Data Spliting Into Train and Testing

In [72]:
X_new

,MeanP_Alpha_F4,MeanP_Beta_F4,MeanP_Theta_F4,MeanP_Delta_F4,MeanP_Gamma_F4,MeanP_Alpha_C4,MeanP_Beta_C4,MeanP_Theta_C4,MeanP_Delta_C4,MeanP_Gamma_C4,...,DAR_C4,DAR_O2,DAR_G,DTR_F4,DTR_O2,DTABR_F4,DTABR_C4,DTABR_O2,DTABR_G,P_G
0,0.00051,0.00071,0.00121,0.01070,0.000240,0.00041,0.00067,0.00061,0.00353,0.000210,...,8.61,25.00,18.20,8.85,18.60,9.77,3.83,8.67,7.42,0.00918
1,0.00040,0.00051,0.00062,0.00539,0.000120,0.00038,0.00052,0.00046,0.00238,0.000100,...,6.26,30.10,16.60,8.69,28.90,6.60,3.16,12.30,7.34,0.00636
2,0.00036,0.00047,0.00059,0.00967,0.000100,0.00036,0.00049,0.00044,0.00312,0.000087,...,8.67,10.50,15.30,16.40,9.58,12.40,4.19,4.62,7.06,0.00628
3,0.00035,0.00047,0.00043,0.00228,0.000110,0.00036,0.00053,0.00040,0.00144,0.000110,...,4.00,3.64,4.72,5.30,3.64,3.30,2.07,1.90,2.42,0.00278
4,0.00033,0.00043,0.00053,0.00436,0.000094,0.00031,0.00043,0.00038,0.00149,0.000081,...,4.81,7.61,8.54,8.23,7.29,6.43,2.53,3.49,4.15,0.00369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108445,0.00031,0.00039,0.00037,0.00101,0.000063,0.00030,0.00038,0.00036,0.00096,0.000060,...,3.20,3.26,3.24,2.73,2.73,1.97,1.94,1.97,1.96,0.00212
108446,0.00042,0.00044,0.00052,0.00426,0.000065,0.00041,0.00043,0.00050,0.00312,0.000062,...,7.61,10.10,9.30,8.19,8.19,5.56,4.31,5.56,5.14,0.00531
108447,0.00040,0.00037,0.00049,0.00116,0.000041,0.00036,0.00033,0.00046,0.00104,0.000037,...,2.89,2.90,2.90,2.37,2.37,2.14,2.17,2.14,2.15,0.00238
108448,0.00051,0.00075,0.00060,0.00508,0.000240,0.00041,0.00056,0.00043,0.00188,0.000140,...,4.59,9.96,8.17,8.47,8.47,4.51,2.38,4.51,3.80,0.00593


In [73]:
y_new

0         4
1         4
2         4
3         4
4         4
         ..
108445    0
108446    0
108447    0
108448    4
108449    4
Name: Sleep Stage, Length: 108450, dtype: int64

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

#Only Training Balancing

In [ ]:
X_train,y_train = Xy_balance(X_train,y_train)




---



# **Training**

##ADABOOST

In [75]:
%%time
from sklearn.ensemble import AdaBoostClassifier

ada_defult = AdaBoostClassifier(random_state=0)
ada_defult.fit(X_train, y_train)
y_pred = ada_defult.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(ada_defult,1,'AdaBoostClassifier')]=accuracy_score(y_test, y_pred)


[[ 127  829  102  544  783]
 [  72 4755 1668  772  603]
 [  11 1021 3119   16   52]
 [  86 1412  161 1435  306]
 [  76  527   74  214 2925]]
              precision    recall  f1-score   support

           0       0.34      0.05      0.09      2385
           1       0.56      0.60      0.58      7870
           2       0.61      0.74      0.67      4219
           3       0.48      0.42      0.45      3400
           4       0.63      0.77      0.69      3816

    accuracy                           0.57     21690
   macro avg       0.52      0.52      0.50     21690
weighted avg       0.54      0.57      0.54     21690

Accurecy:  0.5698939603503919
CPU times: user 18.9 s, sys: 36 ms, total: 19 s
Wall time: 18.9 s




---



---



##Graddient Boosting

In [76]:
%%time
from sklearn.ensemble import GradientBoostingClassifier

gradBoost_default = GradientBoostingClassifier(random_state=0)
gradBoost_default.fit(X_train, y_train)
y_pred = gradBoost_default.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(gradBoost_default,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

[[ 369  883   54  415  664]
 [ 166 5955  852  519  378]
 [   1 1123 3044   14   37]
 [ 187 1026   99 1930  158]
 [ 205  452   33   78 3048]]
              precision    recall  f1-score   support

           0       0.40      0.15      0.22      2385
           1       0.63      0.76      0.69      7870
           2       0.75      0.72      0.73      4219
           3       0.65      0.57      0.61      3400
           4       0.71      0.80      0.75      3816

    accuracy                           0.66     21690
   macro avg       0.63      0.60      0.60     21690
weighted avg       0.65      0.66      0.64     21690

Accurecy:  0.6614107883817427
CPU times: user 8min 12s, sys: 529 ms, total: 8min 13s
Wall time: 8min 11s


###Histogram-Based Gradient Boosting

In [77]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)
hisgradBoost_default.fit(X_train, y_train)
y_pred = hisgradBoost_default.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(hisgradBoost_default,3,'HistGradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

[[ 497  824   38  380  646]
 [ 200 6229  736  402  303]
 [   9  940 3241    7   22]
 [ 165  744   52 2335  104]
 [ 226  293   19   56 3222]]
              precision    recall  f1-score   support

           0       0.45      0.21      0.29      2385
           1       0.69      0.79      0.74      7870
           2       0.79      0.77      0.78      4219
           3       0.73      0.69      0.71      3400
           4       0.75      0.84      0.79      3816

    accuracy                           0.72     21690
   macro avg       0.68      0.66      0.66     21690
weighted avg       0.70      0.72      0.70     21690

Accurecy:  0.71572153065929
CPU times: user 33.5 s, sys: 265 ms, total: 33.7 s
Wall time: 17.6 s




---



---



##Random Forest

In [78]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)
rf_default.fit(X_train, y_train)
y_pred=rf_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_default,4,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

[[ 550  817   33  347  638]
 [ 225 6383  607  378  277]
 [   5  875 3307    6   26]
 [ 159  763   42 2316  120]
 [ 247  320   19   46 3184]]
              precision    recall  f1-score   support

           0       0.46      0.23      0.31      2385
           1       0.70      0.81      0.75      7870
           2       0.83      0.78      0.80      4219
           3       0.75      0.68      0.71      3400
           4       0.75      0.83      0.79      3816

    accuracy                           0.73     21690
   macro avg       0.70      0.67      0.67     21690
weighted avg       0.71      0.73      0.71     21690

Accurecy:  0.7256800368833564
CPU times: user 1min 31s, sys: 83.9 ms, total: 1min 31s
Wall time: 47 s




---



---



##XGB

In [79]:
%%time
import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)
xgb_deafult.fit(X_train.values,y_train.values)
y_pred = xgb_deafult.predict(X_test.values)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_deafult,5,'xgboost')]=accuracy_score(y_test, y_pred)

[[ 263  879   64  453  726]
 [ 130 5833  934  568  405]
 [   2 1214 2949   11   43]
 [ 151 1113  116 1835  185]
 [ 155  464   41   83 3073]]
              precision    recall  f1-score   support

           0       0.38      0.11      0.17      2385
           1       0.61      0.74      0.67      7870
           2       0.72      0.70      0.71      4219
           3       0.62      0.54      0.58      3400
           4       0.69      0.81      0.75      3816

    accuracy                           0.64     21690
   macro avg       0.60      0.58      0.57     21690
weighted avg       0.62      0.64      0.62     21690

Accurecy:  0.6432918395573998
CPU times: user 1min 53s, sys: 108 ms, total: 1min 53s
Wall time: 59.2 s


##KNN

In [80]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)
knn_default.fit(X_train, y_train)
y_pred=knn_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(knn_default,6,'KNeighborsClassifier')]=accuracy_score(y_test, y_pred)

[[ 567  895  121  346  456]
 [ 582 5348 1113  621  206]
 [  41 1281 2820   54   23]
 [ 412 1548  173 1127  140]
 [ 529  485  119  190 2493]]
              precision    recall  f1-score   support

           0       0.27      0.24      0.25      2385
           1       0.56      0.68      0.61      7870
           2       0.65      0.67      0.66      4219
           3       0.48      0.33      0.39      3400
           4       0.75      0.65      0.70      3816

    accuracy                           0.57     21690
   macro avg       0.54      0.51      0.52     21690
weighted avg       0.57      0.57      0.56     21690

Accurecy:  0.5696173351775011
CPU times: user 1min 9s, sys: 3.66 s, total: 1min 13s
Wall time: 55.9 s


##NB

In [81]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()
nb_deafult.fit(X_train, y_train)
y_pred = nb_deafult.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(nb_deafult,7,'GaussianNB')]=accuracy_score(y_test, y_pred)

[[  50 2335    0    0    0]
 [ 121 7749    0    0    0]
 [  47 4172    0    0    0]
 [  28 3372    0    0    0]
 [  46 3770    0    0    0]]
              precision    recall  f1-score   support

           0       0.17      0.02      0.04      2385
           1       0.36      0.98      0.53      7870
           2       0.00      0.00      0.00      4219
           3       0.00      0.00      0.00      3400
           4       0.00      0.00      0.00      3816

    accuracy                           0.36     21690
   macro avg       0.11      0.20      0.11     21690
weighted avg       0.15      0.36      0.20     21690

Accurecy:  0.3595666205624712
CPU times: user 161 ms, sys: 20 ms, total: 181 ms
Wall time: 178 ms


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Support Vector Machines

####SVM

In [83]:
"""%%time
from sklearn import svm

svm = svm.SVC(decision_function_shape='ovo')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(svm,8,'svm.SVC')]=accuracy_score(y_test, y_pred)"""

[[   0 2385    0    0    0]
 [   0 7870    0    0    0]
 [   0 4219    0    0    0]
 [   0 3400    0    0    0]
 [   0 3816    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2385
           1       0.36      1.00      0.53      7870
           2       0.00      0.00      0.00      4219
           3       0.00      0.00      0.00      3400
           4       0.00      0.00      0.00      3816

    accuracy                           0.36     21690
   macro avg       0.07      0.20      0.11     21690
weighted avg       0.13      0.36      0.19     21690

Accurecy:  0.3628400184416782
CPU times: user 19min 55s, sys: 1.23 s, total: 19min 57s
Wall time: 19min 51s


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


####Linear

In [ ]:
"""%%time
from sklearn import svm

LinearSVC = svm.LinearSVC()
LinearSVC.fit(X_train, y_train)
y_pred = LinearSVC.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(LinearSVC,9,'svm.LinearSVC')]=accuracy_score(y_test, y_pred)"""

##Stochastic Gradient Descent

In [84]:
"""%%time
from sklearn.linear_model import SGDClassifier

SGDClassifier = SGDClassifier()
SGDClassifier.fit(X_train, y_train)
y_pred = SGDClassifier.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(SGDClassifier,10,'SGDClassifier')]=accuracy_score(y_test, y_pred)"""

[[   0    0    0    0 2385]
 [   0    0    0    0 7870]
 [   0    0    0    0 4219]
 [   0    0    0    0 3400]
 [   0    0    0    0 3816]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2385
           1       0.00      0.00      0.00      7870
           2       0.00      0.00      0.00      4219
           3       0.00      0.00      0.00      3400
           4       0.18      1.00      0.30      3816

    accuracy                           0.18     21690
   macro avg       0.04      0.20      0.06     21690
weighted avg       0.03      0.18      0.05     21690

Accurecy:  0.17593360995850624
CPU times: user 13.6 s, sys: 78 ms, total: 13.7 s
Wall time: 13.6 s


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Tree algorithms

###DecisionTreeRegressor

In [86]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(dtc,11,'DecisionTreeClassifier')]=accuracy_score(y_test, y_pred)

[[ 601  741   77  406  560]
 [ 721 4960 1016  787  386]
 [  82 1057 2943   77   60]
 [ 431  830   82 1895  162]
 [ 597  440   60  175 2544]]
              precision    recall  f1-score   support

           0       0.25      0.25      0.25      2385
           1       0.62      0.63      0.62      7870
           2       0.70      0.70      0.70      4219
           3       0.57      0.56      0.56      3400
           4       0.69      0.67      0.68      3816

    accuracy                           0.60     21690
   macro avg       0.56      0.56      0.56     21690
weighted avg       0.60      0.60      0.60     21690

Accurecy:  0.596726602120793
CPU times: user 7.24 s, sys: 8.99 ms, total: 7.25 s
Wall time: 7.25 s




---



---



#Result

In [82]:

models=[]

for i in result:
  models.append(i[0])
  print(i[0],i[1]," : ",result[i])
  print("---------------------------------------------------------------")
  print()


AdaBoostClassifier(random_state=0) 1  :  0.5698939603503919
---------------------------------------------------------------

GradientBoostingClassifier(random_state=0) 2  :  0.6614107883817427
---------------------------------------------------------------

HistGradientBoostingClassifier(random_state=0) 2  :  0.71572153065929
---------------------------------------------------------------

RandomForestClassifier(n_jobs=-1, random_state=0) 3  :  0.7256800368833564
---------------------------------------------------------------

XGBClassifier(n_jobs=-1, objective='multi:softprob') 4  :  0.6432918395573998
---------------------------------------------------------------

KNeighborsClassifier(n_jobs=-1) 5  :  0.5696173351775011
---------------------------------------------------------------

GaussianNB() 6  :  0.3595666205624712
---------------------------------------------------------------

